### ADD Brand List of Shopinng Mall

In [ ]:
import sys

sys.path.append("../../")

In [ ]:
# in Jupyter
import re
import sys
import os
from random import randint
from typing import List, Dict, Any, Callable

import asyncio 
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, Page, TimeoutError as PlaywrightTimeoutError 



pw = await async_playwright().start()
browser = await pw.firefox.launch(headless=False)
page = await browser.new_page()


page.set_default_timeout(15000)

async def _load_page( page: Page, url):
    """페이지 로드"""
    await asyncio.sleep(1)
    await page.goto(url)
    await page.wait_for_load_state(state="networkidle")
    await page.wait_for_timeout(1000)
    return page

async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)

### shop

In [ ]:
url = "https://www.sevenstore.com/footwear/sneakers/adidas-cgreenftwwhtmagold-hand-2-sneaker/"

page = await _load_page(page, url)

In [ ]:
async def get_size_info(page:Page) -> List[Dict[str, Any]]:
    await page.is_visible('//div[contains(@class, "size-wrapper")]',timeout=10000)
    size_query = await page.query_selector_all(
        '//div[contains(@class, "size-wrapper")]',
    )
    print(size_query[0])
    await size_query[0].isEnabled()

    size_list = [await s.inner_text() for s in size_query]

    if not size_list:
        return [{"shop_product_size": "-", "kor_product_size": "-"}]

    l = []
    for s in size_list:
        if s.isdigit() and float(s) < 15:
            kor_size = "UK " + s
        else:
            kor_size = s
        l.append({"shop_product_size": s, "kor_product_size": kor_size})

    return l

async def get_product_id(page) -> str:
    product_id_text = await page.query_selector(
        '//meta[contains(@name, "description")]',
    )

    try:
        product_id_text = await product_id_text.get_attribute("content")
        product_id = product_id_text.split(":")[1].replace(" ", "")
    except:
        product_id = "-"

    return product_id.upper()


await get_size_info(page)

await get_product_id(page)


In [ ]:
size_list

In [ ]:
product_id_text = await page.query_selector('//*[@id="detailsPanel"]/div')
product_id_text = await product_id_text.inner_text()

In [ ]:
text = product_id_text.lower()
if "product code" in text :
    product_id = text.split("product code")[1].strip()
else:
    product_id = '-'

product_id.upper()

### kream

In [ ]:
await _load_page(page, "https://kream.co.kr/login")

In [ ]:
async def save_cookies(page):
    # Get cookies from the current page
    cookies = await page.context.cookies()

    # Save cookies to a file or database
    # In this example, cookies are saved to a file named 'cookies.json'
    with open('../../router/dev/kream/cookie/cookies.json', 'w') as file:
        file.write(str(cookies))

await save_cookies(page)

In [ ]:
async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)
await page.goto("https://kream.co.kr")

In [ ]:
brand = await page.query_selector(".wish_count_num")
brand = await brand.inner_text()
int(brand.replace(",",""))

In [ ]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]

In [ ]:
# def _extract_volume_data_from(body_list: List) -> List[List]:
#     """
#     크롤링 결과 리스트에서 거래량 추출

#     Args:
#         body_list (List): 크롤링 결과 리스트

#     Returns:
#         List: 크롤링 결과 리스트로 반환
#     """

#     def preprocess_items(tags) -> List:
#         l = []
#         for tag in tags.find_all(class_="list_txt"):
#             l.append(tag.text.lstrip().rstrip())

#         if "빠른" in l[1]:
#             l = l[:1] + l[1].split(" ")[:1] + [True] + l[2:3].replace("빠른배송", "").strip()
#         else:
#             l = l[:2] + [False] + l[2:3].replace("빠른배송", "").strip()
#         return l

#     return [preprocess_items(tags) for tags in body_list]

# body_list = _extract_volume_data_from(volumes)

In [ ]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]


def _extract_volume_data_from(body_list: List) -> List[List]:
    """
    크롤링 결과 리스트에서 거래량 추출

    Args:
        body_list (List): 크롤링 결과 리스트

    Returns:
        List: 크롤링 결과 리스트로 반환
    """

    def preprocess_items(tags) -> List:
        x = []
        for tag in tags.find_all(class_="list_txt"):
            x.append(tag.text.strip())
        if "빠"in x[1] :
            return [x[0], x[1], True, x[2].replace('빠른배송','').strip()]
        else :
            return [x[0], x[1], False, x[2].replace('빠른배송','').strip()]

    return [preprocess_items(tags) for tags in body_list]

body_list = _extract_volume_data_from(volumes)
body_list

In [5]:
data = [
    {
      "shopProductCardId": 2181,
      "shopProductSize": "7",
      "korProductSize": "255",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "7.5",
      "korProductSize": "260",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "8",
      "korProductSize": "265",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "8.5",
      "korProductSize": "270",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "9",
      "korProductSize": "275",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "9.5",
      "korProductSize": "280",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "10",
      "korProductSize": "285",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "11",
      "korProductSize": "295",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2181,
      "shopProductSize": "11.5",
      "korProductSize": "300",
      "available": True,
      "updatedAt": "2023-11-20T13:30:50"
    },
    {
      "shopProductCardId": 2182,
      "shopProductSize": "8",
      "korProductSize": "265",
      "available": True,
      "updatedAt": "2023-11-20T13:30:54"
    },
    {
      "shopProductCardId": 2182,
      "shopProductSize": "9",
      "korProductSize": "275",
      "available": True,
      "updatedAt": "2023-11-20T13:30:54"
    },
    {
      "shopProductCardId": 2182,
      "shopProductSize": "9.5",
      "korProductSize": "280",
      "available": True,
      "updatedAt": "2023-11-20T13:30:54"
    },
    {
      "shopProductCardId": 2182,
      "shopProductSize": "10",
      "korProductSize": "285",
      "available": True,
      "updatedAt": "2023-11-20T13:30:54"
    },
    {
      "shopProductCardId": 2182,
      "shopProductSize": "11",
      "korProductSize": "295",
      "available": True,
      "updatedAt": "2023-11-20T13:30:54"
    },
    {
      "shopProductCardId": 2183,
      "shopProductSize": "4",
      "korProductSize": "225",
      "available": True,
      "updatedAt": "2023-11-20T13:30:57"
    },
    {
      "shopProductCardId": 2183,
      "shopProductSize": "6",
      "korProductSize": "245",
      "available": True,
      "updatedAt": "2023-11-20T13:30:57"
    },
    {
      "shopProductCardId": 2183,
      "shopProductSize": "8",
      "korProductSize": "265",
      "available": True,
      "updatedAt": "2023-11-20T13:30:57"
    }
  ]

pd.DataFrame(data).to_parquet("test.parquet")